In [52]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.data import wine_data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn import model_selection
import json
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
import random
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold


In [ ]:
def class_fun(feature,profile,group,test_profile,test_group):#
    exp_pre=profile.loc[feature,:]
    test_exp_pre=test_profile.loc[feature,]
    dat2=exp_pre.stack()
    exp=dat2.unstack(0)
    dat3=test_exp_pre.stack()
    test_exp=dat3.unstack(0)
    rf_new = RandomForestClassifier()
    rf_new.fit(exp, group)
    #prediction = rf_new.predict(test_exp)
    #交叉验证，5次准确率
    scores = cross_val_score(rf_new, exp, group, cv=5,scoring='accuracy')
    #print(scores)
    accuracy_value=pd.DataFrame({'accuracy':scores})
    mean_acc=accuracy_value.loc[:,'accuracy'].mean()
    #print(mean_acc)
    #accuracy_selection.append(accuracy_score(prediction, test_group))
    #cvs_selection.append(np.mean(scores))
    ##train_roc
    y_score = rf_new.predict_proba(exp)[:,1]
    fpr,tpr,threshold = metrics.roc_curve(group, y_score)
    roc_auc = metrics.auc(fpr,tpr)
    ##test_roc
    y_score = rf_new.predict_proba(test_exp)[:,1]
    fpr,tpr,threshold = metrics.roc_curve(test_group, y_score)
    roc_auc_test = metrics.auc(fpr,tpr)

    #print(pred_y_rf_new)
    importance=rf_new.feature_importances_
    data = { 'gene' : feature, 'importance' :importance }
    df = pd.DataFrame(data)
    df.loc['ROC']=['ROC',roc_auc]
    df.loc['ROC_test']=['ROC_test',roc_auc_test]
    df.loc['Mean_accuracy']=['Mean_accuracy',mean_acc]

    return df

# In[9]:


def jurge_fun(x):
    if x[7]==x[10]:
        y="a"
    else:
        y="b"
    return y

def findgene_fun(df1,df2):
    module_res=pd.merge(df1,df2,on='gene')
    module_res.loc[:,'type'] = module_res.apply(jurge_fun,1)
    module_res.loc[:,'id'] = module_res.index
    module_res1 =module_res.loc[module_res['type']=='a']
    rank_accuracy=module_res1.groupby(['modeuleType']).apply(lambda t: t[(t['Mean_accuracy']==t['Mean_accuracy'].max())])
    rank_accuracy1=rank_accuracy.drop(columns=['modeuleType'])
    rank_accuracy1.reset_index(inplace=True)
    rank_impor=rank_accuracy1.groupby(['modeuleType']).apply(lambda t: t[(t['Rank']==t['Rank'].max())])
    rank_impor1=rank_impor.drop(columns=['modeuleType','level_1'])
    rank_impor1.reset_index(inplace=True)
    #res_last=pd.merge(module_res,rank_impor1,on='id',how='right')
    #print(res_last)
    res_last=rank_impor1.loc[:,['gene_pair','modeuleType']]
    return res_last.drop_duplicates()

def combination_fun(x,profile,group,test_profile,test_group):
    #gene_all=[]
    all_value=pd.DataFrame({'gene_pair':[],'gene':[],'importance':[],'Rank':[],'roc':[],'roc_test':[],'module_two_genes':[],
                            'iteration1':[],'iteration2':[]})
    module=x.iloc[:,1].drop_duplicates().values.tolist()
    for mm in module:
        #print(mm)
        gene_module=x.loc[x['modeuleType'] != mm].gene.tolist()
        #print(gene_module)
        y=x[x.modeuleType.isin([mm])].gene
        for i in range(0,len(y)-1):
            for j in range(i+1,len(y)):
                gene=[y[i],y[j]]
                gene_all=gene_module+gene
                #print(gene_all)
                class_results=class_fun(gene_all,profile,group,test_profile,test_group)#
                roc=class_results.loc['ROC'].importance
                roc_test=class_results.loc['ROC_test'].importance
                mean_acc=class_results.loc['Mean_accuracy'].importance
                gene_importance=class_results.drop(index=["ROC","ROC_test","Mean_accuracy"])
                gene_importance['Rank'] = gene_importance['importance'].rank(method="average")

                #gene_all=gene_all.append(gene_importance)
                #roc_all.extend([roc])
                valuedf=pd.DataFrame({'gene_pair':'_'.join(gene),'gene':gene_importance.gene,'importance':gene_importance.importance,
                                      'Rank':gene_importance.Rank,
                                          'Mean_accuracy':mean_acc,
                                          'roc':roc,'roc_test':roc_test,'module_two_genes':mm
                                          ,'iteration1':i,'iteration2':j})
                all_value=pd.concat([valuedf,all_value])
    gene_all=findgene_fun(all_value,x)
                #print(gene)
    all_value.to_csv('all_value_com_moremodule.csv')
#print(gene_all)
    return gene_all

# In[ ]:
gene_module=pd.DataFrame(pd.read_csv('dm_inter_gene_module_train_most.csv'
                                      ,header=0,index_col=0))

#read data
dat=pd.DataFrame(pd.read_csv('dm_inter_train_most.csv',header=0,index_col=0))
group=[ 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0]

dat1=pd.DataFrame(pd.read_csv('dm_inter_test_most.csv',header=0,index_col=0))
####group
y_test=[1, 1, 0, 1, 0, 1, 1, 0]


# gene_last=[]
# for i in range(1,100):
#     print(i)
#     gene_all=combination_fun(gene_module,dat,group,dat1,y_test)
    #gene_all=findgene_fun(tt,gene_module)
#     gene_last.append(gene_all)


arg_l = [(gene_module, dat, group, dat1, y_test) for i in range(1000)]

with Pool(48) as pool:
    res=pool.starmap(combination_fun, arg_l)
    pool.close()
    pool.join()

#gene_list = list(np.ravel(res))
#def flat(a):
#    l=[]
#    for i in a:
#        if type(i) is list:
#            for j in i:
#                l.append(j)
#        else:
#            l.append(i)
#    return(l)
#gene_list = flat(res)
#print(gene_list)
#gene_count=Counter(gene_list)
#gene_count_all=pd.DataFrame.from_dict(gene_count,orient='index',columns=['count']).reset_index()

def flat(a):
    l=pd.DataFrame({'gene_pair':[],'modeuleType':[]})
    for i in a:
        l=pd.concat([l,i])
    return(l)

gene_list = flat(res)
print(gene_list)
type(gene_list)


tt=gene_list.groupby('modeuleType').value_counts()
#print(type(tt))
tt=tt.reset_index()
tt.columns = ['moduleType','gene_pair','count']
tt.to_csv('gene_count_all_moremodule.csv')

#gene_count_all.to_csv('gene_count_all_moremodule.csv')


In [106]:
gene_count_all=pd.DataFrame(pd.read_csv('gene_count_all_moremodule.csv',header=0,index_col=0))
gene_count_all.sort_values(by="count",ascending=False,inplace=True)

In [149]:
print(gene_count_all.groupby('moduleType').head(2))

        moduleType       gene_pair  count
70  MEMidnightblue   RNASE6_COL9A3    144
87           METan     HLA-DOB_CD9    144
88           METan  HLA-DOB_CORO2B    119
71  MEMidnightblue     CSTA_GLIPR1    118
0          MEBlack    TGFB1I1_PPBP     96
24         MEGreen     ALPL_BCL2A1     96
1          MEBlack      JUP_ADORA3     96
46       MEMagenta   SLC16A6_UBE2W     96
47       MEMagenta      NFAT5_KRAS     96
25         MEGreen        VNN1_ADM     70


In [152]:
gene_all=['HLA-DOB','RNASE6','CD9','TGFB1I1','JUP','ADORA3','NFAT5','KRAS','PPBP','ALPL','BCL2A1','SLC16A6','UBE2W','COL9A3']#gene_count_all.sort_values(by="count",ascending=False)

In [156]:
test=class_fun(gene_all,dat,group,dat1,y_test)
print(test)

                        gene  importance
0                    HLA-DOB    0.184691
1                     RNASE6    0.058414
2                        CD9    0.048856
3                    TGFB1I1    0.072613
4                        JUP    0.068060
5                     ADORA3    0.066843
6                      NFAT5    0.058124
7                       KRAS    0.102081
8                       PPBP    0.035557
9                       ALPL    0.057354
10                    BCL2A1    0.043681
11                   SLC16A6    0.104561
12                     UBE2W    0.063904
13                    COL9A3    0.035261
ROC                      ROC    1.000000
ROC_test            ROC_test    0.733333
Mean_accuracy  Mean_accuracy    0.904762


In [157]:
feature=gene_all#["ATP6V1D","HLA-DOB"]#
profile=dat
group=group
test_profile=dat1
test_group=y_test

In [172]:
exp_pre=profile.loc[feature,:]
test_exp_pre=test_profile.loc[feature,]
dat2=exp_pre.stack()
exp=dat2.unstack(0)
dat3=test_exp_pre.stack()
test_exp=dat3.unstack(0)
rf_new = RandomForestClassifier()
rf_new.fit(exp, group)
    

RandomForestClassifier()

In [173]:
scores = cross_val_score(rf_new, exp, group, cv=5,scoring='accuracy')
    #print(scores)
accuracy_value=pd.DataFrame({'accuracy':scores})
mean_acc=accuracy_value.loc[:,'accuracy'].mean()
print(mean_acc)
    #accuracy_selection.append(accuracy_score(prediction, test_group))
    #cvs_selection.append(np.mean(scores))
    ##train_roc
y_score = rf_new.predict_proba(exp)[:,1]
fpr,tpr,threshold = metrics.roc_curve(group, y_score)
roc_auc = metrics.auc(fpr,tpr)
    ##test_roc
y_score = rf_new.predict_proba(test_exp)[:,1]
fpr,tpr,threshold = metrics.roc_curve(test_group, y_score)
roc_auc_test = metrics.auc(fpr,tpr)
    
    #print(pred_y_rf_new)
importance=rf_new.feature_importances_
data = { 'gene' : feature, 'importance' :importance }
df = pd.DataFrame(data)
df.loc['ROC']=['ROC',roc_auc]
df.loc['ROC_test']=['ROC_test',roc_auc_test]
df.loc['Mean_accuracy']=['Mean_accuracy',mean_acc]
print(df)

0.9
                        gene  importance
0                    HLA-DOB    0.198743
1                     RNASE6    0.037751
2                        CD9    0.027673
3                    TGFB1I1    0.078291
4                        JUP    0.060165
5                     ADORA3    0.102873
6                      NFAT5    0.071584
7                       KRAS    0.064828
8                       PPBP    0.032999
9                       ALPL    0.067027
10                    BCL2A1    0.048004
11                   SLC16A6    0.101193
12                     UBE2W    0.076031
13                    COL9A3    0.032838
ROC                      ROC    1.000000
ROC_test            ROC_test    0.733333
Mean_accuracy  Mean_accuracy    0.900000


In [174]:
y_score

array([0.53, 0.75, 0.47, 0.32, 0.1 , 0.55, 0.77, 0.71])

In [135]:
test_group

[1, 1, 0, 1, 0, 1, 1, 0]

In [136]:
scoring = {'accuracy': make_scorer(accuracy_score),
               'roc_auc': make_scorer(roc_auc_score),
               'precision': make_scorer(precision_score),
               'recall': make_scorer(recall_score),
               'f1_score': make_scorer(f1_score)}
rf_cv_score = model_selection.cross_validate(rf_new, exp, group, cv=5, scoring=scoring)
print(rf_cv_score)


{'fit_time': array([0.11368012, 0.08275628, 0.0797801 , 0.07751775, 0.08355832]), 'score_time': array([0.01103044, 0.010005  , 0.01028824, 0.01066399, 0.00994277]), 'test_accuracy': array([0.85714286, 0.85714286, 1.        , 0.66666667, 0.83333333]), 'test_roc_auc': array([0.875     , 0.83333333, 1.        , 0.66666667, 0.83333333]), 'test_precision': array([1. , 0.8, 1. , 0.6, 1. ]), 'test_recall': array([0.75      , 1.        , 1.        , 1.        , 0.66666667]), 'test_f1_score': array([0.85714286, 0.88888889, 1.        , 0.75      , 0.8       ])}


In [137]:
acc=(0.85714286+ 0.85714286+0.83333333+ 0.66666667+ 0.83333333)/5
print(acc)
roc_auc=(0.875 +0.83333333+0.83333333+ 0.66666667+ 0.83333333)/5
print(roc_auc)
preci=(1. +0.8+1. + 0.6+1.)/5
print(preci)
recall=(0.75   +1. +0.66666667+1. +0.66666667)/5
print(recall)
f1=(0.85714286+ 0.88888889+ 0.8 + 0.75 + 0.8 )/5
print(f1)

0.80952381
0.8083333319999999
0.8800000000000001
0.8166666680000001
0.81920635


In [30]:
print(y_score)
print(y_test)
print(tpr)
print(fpr)
print(roc_auc)


[0.34 0.79 0.28 0.39 0.05 0.32 0.89 0.7 ]
[1, 1, 0, 1, 0, 1, 1, 0]
[0.  0.2 0.4 0.4 1.  1. ]
[0.         0.         0.         0.33333333 0.33333333 1.        ]
0.8
